In [17]:
import pandas as pd
import numpy as np
from kiteconnect import KiteConnect, KiteTicker
import login
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
import pytz
import pymongo
import threading

In [2]:
# connect to kite api
user_name = "ZT8131"
password = "GoalFi@123"
totp = "EEJXULM7H43NHJ2KNYJJQ7ZK73INQG35"
api_key = "50ck6xue25zuf01z"
api_secret = "oprzecf4hej9hbs6m621viawto8erfyw"

kite = KiteConnect(api_key=api_key)
request_token = login.kiteLogin(user_name, password, totp, api_key)
data = kite.generate_session(request_token, api_secret)
kite.set_access_token(data["access_token"])

In [3]:
instrument_dump = kite.instruments()   # get instruments dump from NSE
instrument_df = pd.DataFrame(instrument_dump)  # dump it to a dataframe
print(instrument_df.columns)

Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange'],
      dtype='object')


In [4]:
def getInstrumentToken(symbol):
    #return none if not found
    if len(instrument_df[(instrument_df['tradingsymbol']==symbol) & (instrument_df['exchange']=='NSE')]['instrument_token'].values)==0:
        return None
    return instrument_df[(instrument_df['tradingsymbol']==symbol) & (instrument_df['exchange']=='NSE')]['instrument_token'].values[0]

In [15]:
#get the list of finserv stocks from instrument dump
bse_stocks=pd.read_csv('bse.csv')
#keep only Financial Services stocks
bse_stocks=bse_stocks[bse_stocks['Sector Name']=='Financial Services']
#reset the index
bse_stocks.reset_index(drop=True,inplace=True)

token_list=[]
#get the security id and check if it is present in the instrument dump with exchange as NSE
for index, row in bse_stocks.iterrows():
    symbol=row['Security Id']
    print(symbol)
    token=getInstrumentToken(symbol)
    print(token)
    if token:
        print(symbol,token)
        token_list.append(token)
    else:
        print('Token not found for symbol:',symbol)
        #remove the row from the dataframe
        bse_stocks.drop(index,inplace=True)

#add token list to the dataframe
bse_stocks['Token']=token_list
#remove if Industry == Mutual Fund Scheme - ETF
bse_stocks=bse_stocks[bse_stocks['Industry']!='Mutual Fund Scheme - ETF']
#reset the index
bse_stocks.reset_index(drop=True,inplace=True)
#set the index to 'Security Id'
bse_stocks.set_index('Security Id',inplace=True)
#first sort accourding to 'Security Id' and then 'Industry'
bse_stocks.sort_values(by=['Industry','Security Id'],inplace=True)
print(bse_stocks)

BAJFINANCE
81153
BAJFINANCE 81153
QUESTCAP
None
Token not found for symbol: QUESTCAP
CRISIL
193793
CRISIL 193793
IFCI
381697
IFCI 381697
SBIN
779521
SBIN 779521
IDBI
377857
IDBI 377857
FGP
None
Token not found for symbol: FGP
PHCAP
None
Token not found for symbol: PHCAP
GFLLIMITED
None
Token not found for symbol: GFLLIMITED
GLFL
None
Token not found for symbol: GLFL
HDFCBANK
341249
HDFCBANK 341249
NDLVENTURE
3724033
NDLVENTURE 3724033
INDLEASE
None
Token not found for symbol: INDLEASE
MARGOFIN
None
Token not found for symbol: MARGOFIN
KOTAKBANK
492033
KOTAKBANK 492033
LICHSGFIN
511233
LICHSGFIN 511233
HYBRIDFIN
None
Token not found for symbol: HYBRIDFIN
MAHSCOOTER
None
Token not found for symbol: MAHSCOOTER
MFSL
548353
MFSL 548353
PEL
617473
PEL 617473
PRIMESECU
3037185
PRIMESECU 3037185
VHL
3044353
VHL 3044353
MPILCORPL
None
Token not found for symbol: MPILCORPL
FEDERALBNK
261889
FEDERALBNK 261889
BAJAJHLDNG
78081
BAJAJHLDNG 78081
SUNRINV
None
Token not found for symbol: SUNRINV
DSINV

In [ ]:
def calculate_moment_score(df):
    if len(df) < 2:
        return np.nan
    # Get the pct change from the first to the last value
    pct_change = (df['close'].iloc[-1] / df['close'].iloc[0] - 1)
    # Get the std of pct change
    pct_change_std = np.std(df['close'].pct_change())
    if pct_change_std == 0:
        return np.nan
    score_value = pct_change / pct_change_std
    return score_value


#fuction to get zscore accourding to each column
def get_zscore(df):
    for column in df.columns:
        # print(df[column])
        df[column] = (df[column] - df[column].mean())/df[column].std()
    return df
    

def get_moment_score(df,date):
        moment_dict = {}
        # print(df.head(10))
        symbollist = df.index.to_list()
        # print(symbollist)
        end_date = datetime.strptime(date, '%Y-%m-%d')
        start_date = end_date - relativedelta(months=6)
        start_date = start_date.strftime('%Y-%m-%d')

        for symbol in symbollist:
            # print(symbol)

            try:
                stock_data = kite.historical_data(getInstrumentToken(symbol), start_date, end_date, "day")
                #make dataframe of the stock data
                stock_data = pd.DataFrame(stock_data)
                data_points = len(stock_data)
                three_month = stock_data.tail(data_points//2)
                data_points = len(three_month)
                one_month = three_month.tail(data_points//3)
                data_points = len(one_month)
                two_week = one_month.tail(data_points//2)
                data_points = len(two_week)
                one_week = stock_data.tail(5)
                three_days = stock_data.tail(3) 


                socre_t = calculate_moment_score(stock_data)
                score_3m = calculate_moment_score(three_month)
                score_1m = calculate_moment_score(one_month)
                score_2w = calculate_moment_score(two_week)
                score_1w = calculate_moment_score(one_week)
                score_3d = calculate_moment_score(three_days)

                # print(socre_t,score_3m,score_1m,score_2w,score_1w,score_3d)

                row={'score_t':socre_t,'score_3m':score_3m,'score_1m':score_1m,'score_2w':score_2w,'score_1w':score_1w,'score_3d':score_3d}
                moment_dict[symbol] = row

            except:
                # print("Error in getting data for symbol: ", symbol)
                moment_dict[symbol] = {"Z_score_t": np.nan, "Z_score_3m": np.nan, "Z_score_1m": np.nan, "Z_score_2w": np.nan, "Z_score_1w": np.nan, "Z_score_3d": np.nan}

        moment_score_results = pd.DataFrame(moment_dict).T
        # print(moment_score_results)
        moment_score_results = get_zscore(moment_score_results)
        return moment_score_results
        # df['Moment Score'] = moment_score_results['Moment_score']

        # return df

# get the moment score for the stocks
moment_score_df = get_moment_score(bse_stocks,'2024-10-01')

In [14]:
bse_stocks['Industry']=bse_stocks['Industry'].str.strip()
bse_stocks['Igroup Name']=bse_stocks['Igroup Name'].str.strip()
moment_score_df['Industry']=bse_stocks['Industry']
moment_score_df['Igroup Name']=bse_stocks['Igroup Name']
print(moment_score_df)


             score_t  score_3m  score_1m  score_2w  score_1w  score_3d  \
ABSLAMC     1.509605  0.677743  0.110140  0.008828  0.432924 -0.074160   
HDFCAMC    -0.071539  0.268376 -0.400188 -0.169610 -0.070519 -0.080270   
IVC        -0.065637 -0.472742 -0.652138  0.579454  0.758195 -0.069931   
NAM-INDIA   0.629020 -0.044455 -0.058498  0.384967  0.231564 -0.070362   
UTIAMC      1.803414  1.201656  0.567798 -2.004701 -4.758622 -0.116387   
...              ...       ...       ...       ...       ...       ...   
MOTILALOFS  1.117523  1.256937 -0.231834 -0.619928 -0.272929 -0.079899   
NUVAMA      0.496991  1.024511 -0.387417 -0.163382 -0.289811 -0.097914   
ONELIFECAP -2.043537 -0.007251  0.034718 -3.673437 -1.732184 -0.092471   
SHAREINDIA -0.529018  0.396074  1.412252  0.896206 -0.190562 -0.078785   
SMCGLOBAL   0.021582 -0.376236  0.671940 -0.843174 -1.984393 -0.095034   

                            Industry      Igroup Name  
ABSLAMC     Asset Management Company  Capital Markets  

In [9]:
#calulate beta for each stock
def get_beta(stock):
    try:
        stock_data = kite.historical_data(getInstrumentToken(stock), from_date=datetime.now() - timedelta(days=60), to_date=datetime.now(), interval='day')
        market_data = kite.historical_data(256265, from_date=datetime.now() - timedelta(days=60), to_date=datetime.now(), interval='day')
        stock_df = pd.DataFrame(stock_data)
        stock_df['Close'] = stock_df['close']
        stock_df['Returns'] = stock_df['Close'].pct_change()

        market_df = pd.DataFrame(market_data)
        market_df['Close'] = market_df['close']
        market_df['Market Returns'] = market_df['Close'].pct_change()

        stock_df = stock_df.dropna()
        stock_df = stock_df.reset_index(drop=True)

        market_df = market_df.dropna()
        market_df = market_df.reset_index(drop=True)

        beta = np.cov(stock_df['Returns'], market_df['Market Returns'])[0][1] / np.var(market_df['Market Returns'])
    except:
        beta = np.nan
    print(stock, beta)
    return beta


#calculate beta for all the stocks
beta_dict={}
for index, row in moment_score_df.iterrows():
    stock=index
    beta=get_beta(stock)
    beta_dict[stock]=beta

ABSLAMC 1.4830387642873284
HDFCAMC 1.0786575773884117
IVC 0.798181822294304
NAM-INDIA 1.0767822328682368
UTIAMC 0.32329539349528313
CAMS 1.4946376897226177
KFINTECH 1.9206196736443426
IEX 1.308901644178905
MCX 0.8902535249278543
HUDCO 2.3828077345829626
IFCI 1.7597748118807253
IREDA 1.6190891673586303
IRFC 1.2712300464590487
PFC 1.9676816943547752
RECLTD 1.839549821685601
TFCILTD 1.4601802879083614
ANANDRATHI 0.3619578268721084
PRUDENT 1.8147832729246465
AGSTRA 1.93071874468064
INFIBEAM 0.2078032819404043
PAYTM 1.711877895567059
POLICYBZR 0.3284855681766679
SUVIDHAA 0.9539442657432419
GICRE 1.8004385501809181
GODIGIT 0.39241602907195605
ICICIGI 0.8663243976718314
NIACL 1.9379761180454487
STARHEALTH 0.5586754810750766
AHL 0.8985781545443512
BAJAJFINSV 1.1864648510763798
BAJAJHLDNG 0.3355854939963358
BFINVEST 1.4421957841184097
CHOICEIN 0.8182541158445866
EDELWEISS 2.9448776440963327
JMFINANCIL 1.6175628635690584
MAXIND 0.9724567901621904
RANEHOLDIN 1.3588631904973925
AADHARHFC 1.5509683

In [ ]:
#make a dataframe of beta
beta_df=pd.DataFrame(beta_dict.items(),columns=['Security Id','Beta'])
#make 'Security Id' as index    
beta_df.set_index('Security Id',inplace=True)

                 Beta
Security Id          
ABSLAMC      1.483039
HDFCAMC      1.078658
IVC          0.798182
NAM-INDIA    1.076782
UTIAMC       0.323295
...               ...
MOTILALOFS   1.692387
NUVAMA       1.598350
ONELIFECAP  -0.171720
SHAREINDIA   1.583963
SMCGLOBAL    0.885439

[183 rows x 1 columns]
             score_t  score_3m  score_1m  score_2w  score_1w  score_3d  \
ABSLAMC     1.509605  0.677743  0.110140  0.008828  0.432924 -0.074160   
HDFCAMC    -0.071539  0.268376 -0.400188 -0.169610 -0.070519 -0.080270   
IVC        -0.065637 -0.472742 -0.652138  0.579454  0.758195 -0.069931   
NAM-INDIA   0.629020 -0.044455 -0.058498  0.384967  0.231564 -0.070362   
UTIAMC      1.803414  1.201656  0.567798 -2.004701 -4.758622 -0.116387   
...              ...       ...       ...       ...       ...       ...   
MOTILALOFS  1.117523  1.256937 -0.231834 -0.619928 -0.272929 -0.079899   
NUVAMA      0.496991  1.024511 -0.387417 -0.163382 -0.289811 -0.097914   
ONELIFECAP -2.043537 -0.

In [ ]:
#calculate alpha score
def calculate_alpha_score(df):
    if len(df) < 2:
        return np.nan
    # Get the pct change from the first to the last value
    pct_change = (df['close'].iloc[-1] / df['close'].iloc[0] - 1)
    #GET THE EXPECTED RETURN USING BETA
    rfr = 0.06
    beta = df['Beta']
    